In [3]:
import torch

In [4]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [5]:
from torch.utils.data import Dataset, DataLoader

In [12]:
class GPTV1(Dataset):
    def __init__(self, txt, tokenizer, max_len, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt, allowed_special={"|endoftext|"})

        for i in range(0, len(token_ids) - max_len, stride):
            ip_chunk = token_ids[i : i + max_len]
            target_chunk = token_ids[i + 1 : i + max_len + 1]
            self.input_ids.append(torch.tensor(ip_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [6]:
def create_dataloader_v1(txt, batch_size=4, max_len=256, stride=128,
                     shuffle=True, drop_last=True, num_workers=0):
    
    tokenizer = tiktoken.get_encoding("gpt2")
    
    dataset = GPTV1(txt, tokenizer, max_len, stride)

    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [8]:
with open("data/the-verdict.txt", "r") as f:
    raw = f.read()

In [10]:
import tiktoken

In [13]:
max_len = 4
dl = create_dataloader_v1(
    raw, batch_size=8, max_len=max_len,
    stride=max_len, shuffle=False
)
data_iter = iter(dl)
inputs, targets = next(data_iter)

In [14]:
print("Toeken ids:\n", inputs)
print('\n\n', inputs.shape)

Toeken ids:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])


 torch.Size([8, 4])


In [15]:
tok_embeddings = token_embedding_layer(inputs)
tok_embeddings.shape

torch.Size([8, 4, 256])

In [17]:
contex_len = max_len
pos_embedding_layer = torch.nn.Embedding(contex_len, output_dim)

In [19]:
pos_embedding = pos_embedding_layer(torch.arange(max_len))
pos_embedding.shape

torch.Size([4, 256])